## Train MRCNN Model from pipeline

### Setup and input parms

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os, sys, math, io, time, gc, platform, pprint
import numpy as np
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')

import mrcnn.utils      as utils
import mrcnn.visualize  as visualize

from datetime           import datetime   
# from mrcnn.config       import Config
# from mrcnn.dataset      import Dataset 
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.coco         import CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
from mrcnn.prep_notebook import mrcnn_coco_train 
from mrcnn.utils        import log, command_line_parser, Paths
                                
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)

input_parms = " --epochs          17 "
input_parms +=" --steps_in_epoch  16"
input_parms +=" --last_epoch       0 "
input_parms +=" --batch_size       1 " 
input_parms +=" --lr           0.001 "
input_parms +=" --val_steps        4 " 
input_parms +=" --mrcnn_logs_dir train_mrcnn_coco_subset " 
input_parms +=" --opt           adam "
input_parms +=" --mrcnn_model   coco "
input_parms +=" --coco_classes   78 79 80 81 82 44 46 47 48 49 50 51 34 35 36 37 38 39 40 41 42 43 10 11 13 14 15 "
input_parms +=" --new_log_folder "
# input_parms +=" --fcn_logs_dir   train_fcn_coco "
# input_parms +=" --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
print(input_parms)

parser = command_line_parser()
args = parser.parse_args(input_parms.split())
utils.display_input_parms(args)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 --epochs          17  --steps_in_epoch  16 --last_epoch       0  --batch_size       1  --lr           0.001  --val_steps        4  --mrcnn_logs_dir train_mrcnn_coco_subset  --opt           adam  --mrcnn_model   coco  --coco_classes   78 79 80 81 82 44 46 47 48 49 50 51 34 35 36 37 38 39 40 41 42 43 10 11 13 14 15  --new_log_folder 

Arguments passed :
--------------------
batch_size                     1
coco_classes                   [78, 79, 80, 81, 82, 44, 46, 47, 48, 49, 50, 51, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 10, 11, 13, 14, 15]
epochs                         17
fcn_arch                       FCN32
fcn_layers                     fcn32+
fcn_logs_dir                   train_fcn
fcn_losses                     fcn_BCE_loss
fcn_model                      last
last_epoch                     0
lr                             0.001
mrcnn_exclude_layers           None
mrcnn_layers               

###  Build Model

In [13]:
mrcnn_model = mrcnn_coco_train(args, verbose =0)

>>> Initialize Paths
 windows  Windows
>>> Initialize ModelBase model 
   Mode      :  training
   Model dir :  F:\models\train_mrcnn_coco_subset
>>> set_log_dir(): model_path:  None
    set_log_dir(): model_path has NOT been provided : None 
                   NewFolder: False  config.NEW_LOG_FOLDER: True 
    set_log_dir(): weight file template (self.checkpoint_path): F:\models\train_mrcnn_coco_subset\mrcnn20181219T1900\mrcnn_{epoch:04d}.h5 
    set_log_dir(): weight file dir      (self.log_dir)        : F:\models\train_mrcnn_coco_subset\mrcnn20181219T1900 
    set_log_dir(): Last completed epoch (self.epoch)          : 0 
>>> ModelBase initialiation complete
>>> ---Initialize MRCNN model, mode:  training

----------------------------
>>> Resnet Graph 
----------------------------
     Input_image shape : (?, 1024, 1024, 3)

>>> Feature Pyramid Network (FPN) Graph 

>>> RPN Layer 

>>> RPN Outputs  <class 'list'>
      rpn_class_logits/rpn_class_logits:0
      rpn_class/rpn_class:0
 

    Weights file loaded: F:\PretrainedModels\mask_rcnn_coco.h5 
MRCNN  MODEL Load weight file COMPLETE 


In [6]:
# del dataset_train, dataset_val, train_generator, val_generator
from mrcnn.prep_notebook import prep_coco_dataset
dataset_train, train_generator = prep_coco_dataset(['train' , 'val35k'],  mrcnn_model.config, generator = True, shuffle = False,load_coco_classes=args.coco_classes, loadAnns='active_only')
dataset_val  , val_generator   = prep_coco_dataset(['minival'],  mrcnn_model.config, generator = True, shuffle = False,load_coco_classes=args.coco_classes, loadAnns='active_only')

COCO loading annotations file F:\MLDatasets\coco2014\annotations/instances_train2014.json  into memory...
Done (t=20.63s)
creating index...
index created!
 load subset of classes:  [78, 79, 80, 81, 82, 44, 46, 47, 48, 49, 50, 51, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 10, 11, 13, 14, 15]
 image dir            :  F:\MLDatasets\coco2014\train2014
 json_path_dir        :  F:\MLDatasets\coco2014\annotations/instances_train2014.json
 number of images     :  40400
 image_ids[:10]       :  [262146, 524291, 9, 393227, 393230, 131087, 393241, 393242, 131099, 393251]
 image_ids[1000:1010] :  [396150, 265080, 396157, 2951, 396172, 134032, 134033, 527250, 2963, 2964]
COCO loading annotations file F:\MLDatasets\coco2014\annotations/instances_valminusminival2014.json  into memory...
Done (t=8.11s)
creating index...
index created!
 load subset of classes:  [78, 79, 80, 81, 82, 44, 46, 47, 48, 49, 50, 51, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 10, 11, 13, 14, 15]
 image dir            :  F:\MLDatase

In [5]:
print(type(dataset_train), type(train_generator))
print(len(dataset_train.image_info))
print(type(dataset_val), type(val_generator))
print(len(dataset_val.image_info))
print(len(dataset_train.image_ids), len(dataset_val.image_info))
# dataset_train.display_active_class_info()
# print(dataset_train.class_from_source_map)
# print(dataset_train.class_info)

<class 'mrcnn.coco.CocoDataset'> <class 'generator'>
118287
<class 'mrcnn.coco.CocoDataset'> <class 'generator'>
5000
118287 5000


### Load saved weights

In [7]:
##------------------------------------------------------------------------------------
## Load Mask RCNN Model Weight file
##------------------------------------------------------------------------------------
# exclude_list = ["mrcnn_class_logits"]
#load_model(model, init_with = args.model)   
exclude_list = []
mrcnn_model.load_model_weights(init_with = args.mrcnn_model, exclude = exclude_list)   

-----------------------------------------------
 Load Model with init parm: [ coco ]
 Exclude layers: 
-----------------------------------------------
 ---> coco : F:\PretrainedModels\mask_rcnn_coco.h5
>>> load_weights() from : F:\PretrainedModels\mask_rcnn_coco.h5
    Weights file loaded: F:\PretrainedModels\mask_rcnn_coco.h5 
MRCNN  MODEL Load weight file COMPLETE 


In [10]:

mrcnn_model.config.display()  
# mrcnn_model.display_layer_info()
print()
print("Testing Dataset Image Count: {}".format(len(dataset_train.image_ids)))
print("Testing Dataset Class Count: {}".format(dataset_train.num_classes))


Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_DATASET_PATH              F:\MLDatasets\coco2014
COCO_MODEL_PATH                F:\PretrainedModels\mask_rcnn_coco.h5
DETECTION_MAX_INSTANCES        200
DETECTION_MIN_CONFIDENCE       0.1
DETECTION_NMS_THRESHOLD        0.3
DETECTION_PER_CLASS            200
DIR_DATASET                    F:\MLDatasets
DIR_PRETRAINED                 F:\PretrainedModels
DIR_TRAINING                   F:\models
EARLY_STOP_MIN_DELTA           0.0001
EARLY_STOP_PATIENCE            100
EPOCHS_TO_RUN                  17
FCN_INPUT_SHAPE                [256 256]
GPU_COUNT                      1
HEATMAP_SCALE_FACTOR           4
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADD

### Train mrcnn model : `train()`

In [7]:
mrcnn_model.config.LAST_EPOCH_RAN  = 0
mrcnn_model.config.EPOCHS_TO_RUN   = 1
mrcnn_model.config.STEPS_PER_EPOCH = 2
mrcnn_model.config.SYSOUT = 'screen'
print('    last epoch ran : ', mrcnn_model.config.LAST_EPOCH_RAN)
print('    epochs to run  : ', mrcnn_model.config.EPOCHS_TO_RUN)
print('    steps per epoch: ', mrcnn_model.config.STEPS_PER_EPOCH)
print('    learning rate  : ', mrcnn_model.config.LEARNING_RATE)
print('    momentum       : ', mrcnn_model.config.LEARNING_MOMENTUM)
print('    weight decay   : ', mrcnn_model.config.WEIGHT_DECAY)


    last epoch ran :  0
    epochs to run  :  1
    steps per epoch:  2
    learning rate  :  0.001
    momentum       :  0.9
    weight decay   :  0.0002


In [8]:
##----------------------------------------------------------------------------------------------
## Train the head branches
## Passing layers="heads" freezes all layers except the head layers. 
## You can also pass a regular expression to select which layers to train by name pattern.
##----------------------------------------------------------------------------------------------

train_layers = [ 'mrcnn', 'fpn','rpn']   ## equivalent to "heads"
loss_names   = [ "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
# train_layers = [ 'mrcnn']
# loss_names   = [ "mrcnn_class_loss", "mrcnn_bbox_loss"]

mrcnn_model.epoch = mrcnn_model.config.LAST_EPOCH_RAN
# mrcnn_model.config.LEARNING_RATE   = 0.001
# mrcnn_model.config.STEPS_PER_EPOCH = config.STEPS_PER_EPOCH
mrcnn_model.train(dataset_train, 
            dataset_val, 
            learning_rate = mrcnn_model.config.LEARNING_RATE, 
            epochs_to_run = mrcnn_model.config.EPOCHS_TO_RUN,
            layers = train_layers,
            losses = loss_names)                  


['mrcnn', 'fpn', 'rpn']
['(mrcnn\\_.*)', '(fpn\\_.*)', '(rpn\\_.*)']
layers regex : (mrcnn\_.*)|(fpn\_.*)|(rpn\_.*)
type train_dataset: <class 'mrcnn.coco.CocoDataset'>
type val_dataset: <class 'mrcnn.coco.CocoDataset'>
    learning rate :  0.001
    momentum      :  0.9


Selecting layers to train
-------------------------
Layer    Layer Name               Layer Type
   0  input_image            (InputLayer          )   ............................no weights to train ]
   1  zero_padding2d_1       (ZeroPadding2D       )   ............................no weights to train ]
   2  conv1                  (Conv2D              )   ............................not a layer we want to train ]
   3  bn_conv1               (BatchNorm           )   ............................not a layer we want to train ]
   4  activation_1           (Activation          )   ............................no weights to train ]
   5  max_pooling2d_1        (MaxPooling2D        )   ............................no weight

Keras model.losses : 
---------------------
   Tensor("Mean:0", shape=(1, 1), dtype=float32)    name: Mean:0
   Tensor("Mean_2:0", shape=(1, 1), dtype=float32)    name: Mean_2:0
   Tensor("Mean_1:0", shape=(1, 1), dtype=float32)    name: Mean_1:0
   Tensor("Mean_3:0", shape=(1, 1), dtype=float32)    name: Mean_3:0

Keras_model._losses:
---------------------
   Tensor("Mean:0", shape=(1, 1), dtype=float32)    name: Mean:0
   Tensor("Mean_1:0", shape=(1, 1), dtype=float32)    name: Mean_1:0
   Tensor("Mean_2:0", shape=(1, 1), dtype=float32)    name: Mean_2:0
   Tensor("Mean_3:0", shape=(1, 1), dtype=float32)    name: Mean_3:0

Keras_model._per_input_losses:
------------------------------
   None    name: <class 'NoneType'>
{   None: [   <tf.Tensor 'Mean:0' shape=(1, 1) dtype=float32>,
              <tf.Tensor 'Mean_1:0' shape=(1, 1) dtype=float32>,
              <tf.Tensor 'Mean_2:0' shape=(1, 1) dtype=float32>,
              <tf.Tensor 'Mean_3:0' shape=(1, 1) dtype=float32>]}

L2 Regula

D:\Program Files\Anaconda3\envs\TF\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
D:\Program Files\Anaconda3\envs\TF\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
2/2 [==============================] - 180s 90s/step - loss: 4.0594 - rpn_class_loss: 0.0297 - rpn_bbox_loss: 0.2216 - mrcnn_class_loss: 2.2579 - mrcnn_bbox_loss: 1.5503 - val_loss: 4.6527 - val_rpn_class_loss: 0.0257 - val_rpn_bbox_loss: 1.2032 - val_mrcnn_class_loss: 2.6875 - val_mrcnn_bbox_loss: 0.7363

Epoch 00001: val_loss improved from inf to 4.65267, saving model to F:\models\train_mrcnn_coco_subset\mrcnn20181218T1931\mrcnn_0001.h5
Final : self.epoch 1   epochs 1


### Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_2500.h5")
model.keras_model.save_weights(model_path)